# Kerberoasting-T1558.003 TH Notebook

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import timedelta 
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 50)

In [3]:
pd.set_option("max_colwidth", None)

In [4]:
spark = SparkSession.builder \
    .appName("Kerberoasting") \
    .master("spark://helk-spark-master:7077") \
    .config("spark.executor.memory","5g") \
    .enableHiveSupport() \
    .getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

In [5]:
es_reader = (spark.read.format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.nodes","")
    .option("es.nodes.wan.only","true")
    .option("es.net.http.auth.user","elastic"))

In [6]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-2020.03.*")
security_df = es_reader.load("logs-endpoint-winevent-security-2020.03.*")
winevent_etw_df = es_reader.load("logs-endpoint-winevent-etw-2020.03.*")

##### Filter the suspicious Ldap queries by SilkETW sensor  

In [7]:
servicePrincipalName_df = winevent_etw_df.filter( (winevent_etw_df.ProviderName == "Microsoft-Windows-LDAP-Client") & (winevent_etw_df.SearchFilter.rlike("(?i)serviceprincipalname=*")) ).groupby(winevent_etw_df.PID,winevent_etw_df.computer_name).agg(F.collect_set("SearchFilter").alias("SearchFilter"),F.collect_set("AttributeList").alias("AttributeList"),F.collect_set("DistinguishedName").alias("DistinguishedName"),F.min("event.created").alias("min_time_etw"),F.max("event.created").alias("max_time_etw"))

##### Filter Ldap communication by Sysmon sensor

In [8]:
ldap_communication = sysmon_df.where( (sysmon_df.event_id == 3) & (sysmon_df.dst_port == 389) ).groupby(sysmon_df.process_guid,sysmon_df.process_id, sysmon_df.host_name).agg(F.first(F.col("src_ip_addr")).alias("src_ip_addr_ldapSeesion"),F.first(F.col("dst_ip_addr")).alias("dst_ip_addr_ldapSeesion"),F.first(F.col("src_port")).alias("src_port_ldapSeesion"),F.first(F.col("dst_port")).alias("dst_port_ldapSeesion"),F.first(F.col("user_account")).alias("user_account"),F.min('@timestamp').alias("min_time_communication_ldap"),F.max('@timestamp').alias("max_time_communication_ldap"))

##### Join between suspicious Ldap queries with the process who generated them

In [9]:
servicePrincipalName_join_sysmon_df = ldap_communication.join(servicePrincipalName_df, (ldap_communication.process_id == servicePrincipalName_df.PID) & 
                                      (ldap_communication.host_name == servicePrincipalName_df.computer_name),how = "inner")

##### Remove the duplicate fields

In [10]:
servicePrincipalName_join_sysmon_df = servicePrincipalName_join_sysmon_df.drop("PID").drop("computer_name")

##### Sysmon 1 - Process creation

In [11]:
event_1_sysmon = sysmon_df.where(sysmon_df.event_id == '1').groupby(sysmon_df.process_guid, sysmon_df.host_name).agg(F.first("process_name").alias("process_name"),F.first("process_path").alias("process_path"),F.first("process_command_line").alias("process_command_line"),F.first("process_parent_name").alias("process_parent_name"),F.first("process_parent_path").alias("process_parent_path"),F.first("process_parent_command_line").alias("process_parent_command_line"),F.first("hash_sha1").alias("hash_sha1"))

##### Sysmon event 11 - created file

In [12]:
event_11_sysmon = sysmon_df.where(sysmon_df.event_id == '11').groupby(sysmon_df.process_guid, sysmon_df.host_name).agg(F.collect_set("file_name").alias("files_created"),F.min('@timestamp').alias("min_time_files"),F.max('@timestamp').alias("max_time_files"))

##### Sysmon event 12 - Registry key and value create and delete 

In [13]:
event_12_sysmon = sysmon_df.where(sysmon_df.event_id == '12').groupby(sysmon_df.process_guid, sysmon_df.host_name).agg(F.collect_set("registry_key_path").alias("registry_createdDeleted"),F.min('@timestamp').alias("mintime_registry_createdDeleted"),F.max('@timestamp').alias("maxtime_registry_createdDeleted"))

##### Sysmon event 13 - Registry value modifications

In [14]:
event_13_sysmon = sysmon_df.where(sysmon_df.event_id == '13').groupby(sysmon_df.process_guid, sysmon_df.host_name).agg(F.collect_set("registry_key_path").alias("registry_valueSet"),F.min('@timestamp').alias("mintime_registry_valueSet"),F.max('@timestamp').alias("maxtime_registry_valueSet"))

##### Sysmon event 14 - Registry key and value rename

In [15]:
event_14_sysmon = sysmon_df.where(sysmon_df.event_id == '14').groupby(sysmon_df.process_guid, sysmon_df.host_name).agg(F.collect_set("registry_key_path").alias("registry_rename"),F.min('@timestamp').alias("mintime_registry_rename"),F.max('@timestamp').alias("maxtime_registry_rename"))

##### Filter Kerberos communication by Sysmon sensor

In [16]:
kerb_communication = sysmon_df.where( (sysmon_df.event_id == 3) & (sysmon_df.dst_port == 88) ).select(sysmon_df.process_guid,sysmon_df.process_id, sysmon_df.host_name,F.col('@timestamp'),sysmon_df.src_ip_addr,sysmon_df.dst_ip_addr, sysmon_df.src_port, sysmon_df.dst_port).withColumn('seesionMetadata', F.concat(F.col('src_ip_addr'),F.lit('|'), F.col('dst_ip_addr'),F.lit('|'), F.col('src_port'),F.lit('|'), F.col('dst_port')))

##### Drop duplicates

In [17]:
kerb_communication = kerb_communication.drop_duplicates()

In [18]:
kerb_communication_concatSession = kerb_communication.groupby(kerb_communication.process_guid, kerb_communication.host_name).agg(F.collect_set("seesionMetadata").alias("seesionMetadata"))

##### Join between the process that generated suspicious Ldap queries and correlate with other events

In [19]:
kerb_join_1 = event_1_sysmon.join(servicePrincipalName_join_sysmon_df, on = ['process_guid','host_name'] ,how='right')

In [20]:
kerb_join_2 = event_11_sysmon.join(kerb_join_1, on = ['process_guid','host_name'] ,how='right')

In [21]:
kerb_join_3 = event_12_sysmon.join(kerb_join_2, on = ['process_guid','host_name'] ,how='right')

In [22]:
kerb_join_4 = event_13_sysmon.join(kerb_join_3, on = ['process_guid','host_name'] ,how='right')

In [23]:
kerb_join_5 = event_14_sysmon.join(kerb_join_4, on = ['process_guid','host_name'] ,how='right')

In [24]:
kerb_join_6 = kerb_communication_concatSession.join(kerb_join_5, on = ['process_guid','host_name'] ,how='right')

##### Extract the sessions that made Kerberos traffic 

In [25]:
df_explode = kerb_join_6.withColumn("seesionMetadataExplode", F.explode(kerb_join_6.seesionMetadata))

In [26]:
df_split = df_explode.withColumn("_tmp", F.split(F.col("seesionMetadataExplode"), "\|"))

In [27]:
df_split2 = df_split.withColumn("ip_src_kerb" , F.col("_tmp").getItem(0)).withColumn( "ip_dst_kerb" , F.col("_tmp").getItem(1)).withColumn( "port_src_kerb", F.col("_tmp").getItem(2)).withColumn("port_dst_kerb" , F.col("_tmp").getItem(3)).drop("_tmp").drop("seesionMetadataExplode").drop("seesionMetadata")

##### Event 4679 - request TGS by Security log events from DC 

In [28]:
kerberos_logs = security_df.filter( security_df.event_id == "4769").groupby(security_df.IPv4, security_df.src_port).agg(F.collect_set("service_ticket_name").alias("service_ticket_name_requests_account_by_sessions") , F.collect_set("service_ticket_id").alias("service_ticket_id") , F.collect_set("ticket_encryption_type").alias("ticket_encryption_type") ,F.min("@timestamp").alias("requests_service_account_min_time"),F.max("@timestamp").alias("requests_service_account_max_time"))

##### Join between the Kerberos traffic with the suspicious process who generate event 4679 TGS request

In [29]:
kerb_join_6 = kerberos_logs.join(df_split2, (kerberos_logs.IPv4 == df_split2.ip_src_kerb) & (kerberos_logs.src_port == df_split2.port_src_kerb)  ,how='inner')

In [30]:
final_results = kerb_join_6.drop('IPv4').drop('src_port')

In [31]:
final_results = final_results.toPandas()

In [32]:
final_results[["service_ticket_name_requests_account_by_sessions","service_ticket_id","ticket_encryption_type","process_guid","host_name","process_id","process_name","process_path","process_command_line","hash_sha1","user_account","SearchFilter","DistinguishedName","src_ip_addr_ldapSeesion","dst_ip_addr_ldapSeesion","src_port_ldapSeesion","dst_port_ldapSeesion","ip_src_kerb","ip_dst_kerb","port_src_kerb","port_dst_kerb","files_created","registry_createdDeleted","registry_valueSet","registry_rename","requests_service_account_min_time","requests_service_account_max_time","min_time_communication_ldap","max_time_communication_ldap","min_time_etw","max_time_etw"]]

,service_ticket_name_requests_account_by_sessions,service_ticket_id,ticket_encryption_type,process_guid,host_name,process_id,process_name,process_path,process_command_line,hash_sha1,user_account,SearchFilter,DistinguishedName,src_ip_addr_ldapSeesion,dst_ip_addr_ldapSeesion,src_port_ldapSeesion,dst_port_ldapSeesion,ip_src_kerb,ip_dst_kerb,port_src_kerb,port_dst_kerb,files_created,registry_createdDeleted,registry_valueSet,registry_rename,requests_service_account_min_time,requests_service_account_max_time,min_time_communication_ldap,max_time_communication_ldap,min_time_etw,max_time_etw
0,[apacheservice],[S-1-5-21-459946217-3195776219-3473239022-1111],[0x17],73308e2a-0b13-5e72-0000-001052d65300,win10.research.com,3328,rubeus.exe,c:\users\user1\appdata\roaming\microsoft\windows\templates\rubeus.exe,"""c:\users\user1\appdata\roaming\microsoft\windows\templates\rubeus.exe"" kerberoast /usetgtdeleg /outfile:hashes_nt.txt",862241E2DBFF4384BCB61029D34585CBE1F9C39E,research\user1,[(&(samAccountType=805306368)(servicePrincipalName=*)(!samAccountName=krbtgt)(!(UserAccountControl:1.2.840.113556.1.4.803:=2)))],"[DC=research,DC=com]",10.169.232.12,10.169.232.11,49902,389,10.169.232.12,10.169.232.11,49904,88,"[c:\users\user1\appdata\roaming\microsoft\windows\templates\hashes_nt.txt, c:\users\user1\appdata\local\microsoft\windows\schcache\research.com.sch]","[HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\ADs\Providers\LDAP\CN=Aggregate,CN=Schema,CN=Configuration,DC=research,DC=com\File, HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap, HKLM\System\CurrentControlSet\Services\Tcpip\Parameters, HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\ADs\Providers\LDAP\CN=Aggregate,CN=Schema,CN=Configuration,DC=research,DC=com]","[HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\AutoDetect, HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\IntranetName, HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\ADs\Providers\LDAP\CN=Aggregate,CN=Schema,CN=Configuration,DC=research,DC=com\File, HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProxyBypass, HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\UNCAsIntranet, HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\ADs\Providers\LDAP\CN=Aggregate,CN=Schema,CN=Configuration,DC=research,DC=com\Time]",None,2020-03-18 11:50:44.545,2020-03-18 11:50:44.545,2020-03-18 11:50:45.243,2020-03-18 11:50:45.243,2020-03-18 11:50:46.700,2020-03-18 11:50:46.700
1,[sqlservice],[S-1-5-21-459946217-3195776219-3473239022-1110],[0x17],73308e2a-0b13-5e72-0000-001052d65300,win10.research.com,3328,rubeus.exe,c:\users\user1\appdata\roaming\microsoft\windows\templates\rubeus.exe,"""c:\users\user1\appdata\roaming\microsoft\windows\templates\rubeus.exe"" kerberoast /usetgtdeleg /outfile:hashes_nt.txt",862241E2DBFF4384BCB61029D34585CBE1F9C39E,research\user1,[(&(samAccountType=805306368)(servicePrincipalName=*)(!samAccountName=krbtgt)(!(UserAccountControl:1.2.840.113556.1.4.803:=2)))],"[DC=research,DC=com]",10.169.232.12,10.169.232.11,49902,389,10.169.232.12,10.169.232.11,49903,88,"[c:\users\user1\appdata\roaming\microsoft\windows\templates\hashes_nt.txt, c:\users\user1\appdata\local\microsoft\windows\schcache\research.com.sch]","[HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\ADs\Providers\LDAP\CN=Aggregate,CN=Schema,CN=Configuration,DC=research,DC=com\File, HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap, HKLM\System\CurrentControlSet\Services\Tcpip\Parameters, HKU\S-1-5-21-459946217-3195776219-3473239022-1112\Software\Microsoft\ADs\P